In [ ]:
import os
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/feedback_analysis/data/UserFeedbackData.csv')
df.head(3)

,review_id,content,score,TU_count,app_id,app_name,RC_ver
0,1_1,"Ever since the update, there's a weird glitch ...",3,22930,com.zhiliaoapp.musically,TikTok,29.6.4
1,1_2,Don't believe the news!!! You can absolutely c...,5,18518,com.zhiliaoapp.musically,TikTok,29.6.4
2,1_3,Great app. Too many ads. If you saw a video an...,4,3467,com.zhiliaoapp.musically,TikTok,29.8.4


In [ ]:
# !pip install sentence_transformers
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
ver = '29.8.4'
app_name = 'TikTok'
bad_scores = (1,2)
decent_scores = (3,4)


bad_score_df = df[(df['app_name'] == app_name) & (df['RC_ver'] == ver) & (df['score'].isin(bad_scores))]
bad_reviews = bad_score_df['content'].tolist()
decent_score_df = df[(df['app_name'] == app_name) & (df['RC_ver'] == ver)& (df['score'].isin(decent_scores))]
decent_reviews = decent_score_df['content'].tolist()
full_app_df = df[(df['app_name'] == app_name) & (df['RC_ver'] == ver)]
all_reviews = full_app_df['content'].tolist()

print(len(bad_reviews), len(decent_reviews), len(all_reviews))

37 29 80


In [ ]:
# #Extract Embeddings first

# from sentence_transformers import SentenceTransformer
# import pickle

# model = SentenceTransformer('all-MiniLM-L6-v2')
# embeddings = model.encode(documents, show_progress_bar=True)

# with open('/content/drive/MyDrive/feedback_analysis/data/UserFeedbackEmbeddings.pkl', 'wb') as f:
#     pickle.dump(embeddings, f)


In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
custom_stopwords = stopwords.words('english') + [
    "tik", "tok", "app", "tiktok", 'tictok', "just", "the", "it", "and", "to", "my",
    "video", "videos", "use", "using", "apps", "please", "like", "really", "much",
    "want", "need", "one", "would", "get", "make", "even", "still", "also", "well",
]

def bertopic(documents: list, custom_stopwords: list):
  # Step 1 - Extract embeddings
  embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
  # Step 2 - Reduce dimensionality
  umap_model = UMAP(n_neighbors=3, n_components=5, min_dist=0.0, metric='cosine')
  # Small neighbors to capture small neighborhoods well. We'd want a bit small, with min_dist=0 so that even just a few review mention an issue, it would be captured.
  # Step 3 - Cluster reduced embeddings
  hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
  # Step 4 - Tokenize topics
  vectorizer_model = CountVectorizer(stop_words=custom_stopwords)
  # Step 5 - Create topic representation
  ctfidf_model = ClassTfidfTransformer()
  # Step 6 - (Optional) Fine-tune topic representations with
  # a `bertopic.representation` model
  representation_model = KeyBERTInspired()

  # All steps together
  topic_model = BERTopic(
    embedding_model=embedding_model,          # Step 1 - Extract embeddings
    umap_model=umap_model,                    # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
    representation_model=representation_model  # Step 6 - (Optional) Fine-tune topic representations
  )

  topics, probs = topic_model.fit_transform(documents)
  return topic_model, topics, probs

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
topic_model, topics, probs = bertopic(all_reviews, custom_stopwords)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,32,0_screen_watch_reinstall_issues,"[screen, watch, reinstall, issues, network, wi...",[Tik Tok best And Nice App 100 workings Amazin...
1,1,22,1_youtube_review_experience_reasons,"[youtube, review, experience, reasons, enterta...","[It's an awesome app, imo not as good as youtu..."
2,2,14,2_android_glitching_screen_phone,"[android, glitching, screen, phone, settings, ...","[It's good app, but for Android it freezes and..."
3,3,6,3_slideshow_slideshows_disappearing_bug,"[slideshow, slideshows, disappearing, bug, gli...",[Stop changing the way slideshows work. I don'...
4,4,6,4_account_profile_accounts_reinstalling,"[account, profile, accounts, reinstalling, iss...","[new account. can't set up a name, can't set u..."


In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_documents(all_reviews)

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
topic_model.topic_labels_

{-1: '-1_settings_preferences_icons_phone',
 0: '0_pros_content_platform_youtube',
 1: '1_youtube_bug_slideshow_glitch',
 2: '2_hacked_account_permissions_privacy',
 3: '3_android_glitching_screen_phone',
 4: '4_issues_issue_loading_wifi',
 5: '5_followers_unfollows_profile_account'}

In [ ]:
#Save topics and probs
import pickle
import numpy as np
res = np.stack((np.array(topics).astype(np.int8), np.array(probs)), axis = 0)
print(res.shape)

with open('/content/drive/MyDrive/feedback_analysis/data/topics.pkl', 'wb') as f:
    pickle.dump(topics, f)

(2, 80)


In [ ]:
cur_df = topic_model.get_topic_info()
cur_df.to_csv('/content/drive/MyDrive/feedback_analysis/data/sample_result.csv', index = 0)
cur_df

,Topic,Count,Name,Representation,Representative_Docs
0,-1,11,-1_settings_preferences_icons_phone,"[settings, preferences, icons, phone, annoy, a...","[My phone is in English, settings and preferen..."
1,0,15,0_pros_content_platform_youtube,"[pros, content, platform, youtube, personally,...",[Great app. Too many ads. If you saw a video a...
2,1,14,1_youtube_bug_slideshow_glitch,"[youtube, bug, slideshow, glitch, ios, recordi...","[Good app, but there's a glitch that I've had ..."
3,2,13,2_hacked_account_permissions_privacy,"[hacked, account, permissions, privacy, reinst...",[Tiktok is sketchy w/ collecting data. You can...
4,3,12,3_android_glitching_screen_phone,"[android, glitching, screen, phone, filters, s...","[app used to be entertaining, now it's just an..."
5,4,9,4_issues_issue_loading_wifi,"[issues, issue, loading, wifi, network, fix, t...",[Request! It's a really great app! I have usin...
6,5,6,5_followers_unfollows_profile_account,"[followers, unfollows, profile, account, follo...","[new account. can't set up a name, can't set u..."
